In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

In [9]:
__name__ = "RNN"

# Model

In [10]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    return model


In [11]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                try:
                    res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                except Exception as e:
                    print(e)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [12]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [13]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
model = getModel()
model.fit(X_train, y_train, epochs=400, callbacks=[tb_callback])

Epoch 1/400
3/3 [==============================] - 5s 63ms/step - loss: 0.6920 - categorical_accuracy: 0.5579
Epoch 2/400
3/3 [==============================] - 0s 46ms/step - loss: 0.6832 - categorical_accuracy: 0.5579
Epoch 3/400
3/3 [==============================] - 0s 38ms/step - loss: 0.6622 - categorical_accuracy: 0.7684
Epoch 4/400
3/3 [==============================] - 0s 38ms/step - loss: 0.6057 - categorical_accuracy: 0.7895
Epoch 5/400
3/3 [==============================] - 0s 36ms/step - loss: 0.4490 - categorical_accuracy: 0.8000
Epoch 6/400
3/3 [==============================] - 0s 43ms/step - loss: 0.9654 - categorical_accuracy: 0.7789
Epoch 7/400
3/3 [==============================] - 0s 49ms/step - loss: 0.6517 - categorical_accuracy: 0.8316
Epoch 8/400
3/3 [==============================] - 0s 45ms/step - loss: 0.4702 - categorical_accuracy: 0.8000
Epoch 9/400
3/3 [==============================] - 0s 55ms/step - loss: 0.4944 - categorical_accuracy: 0.8316
Epoch 10/4

In [15]:

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            48896     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dense_7 (Dense)             (None, 2)                

In [16]:
model.save(f"{modelPath}/{__name__}/test.h5")